In [ ]:
%%sh
pygmentize sklearn-boston-housing.py

In [2]:
import sagemaker

sess   = sagemaker.Session()
bucket = sess.default_bucket()                     
prefix = 'sklearn-boston-housing'

training = sess.upload_data(path='housing.csv', key_prefix=prefix + "/training")
output   = 's3://{}/{}/output/'.format(bucket,prefix)
print(training)
print(output)

s3://sagemaker-us-east-1-613904931467/sklearn-boston-housing/training/housing.csv
s3://sagemaker-us-east-1-613904931467/sklearn-boston-housing/output/


In [5]:
from sagemaker.sklearn import SKLearn

role = sagemaker.get_execution_role()

sk = SKLearn(entry_point='sklearn-boston-housing.py',
             role=role,
             train_instance_count=1, 
             train_instance_type='ml.m5.large',
             output_path=output,
             hyperparameters={
                  'normalize': True,
                  'test-size': 0.1
              }
)

sk.fit({'training':training})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-06-12 08:23:47 Starting - Starting the training job...
2020-06-12 08:23:49 Starting - Launching requested ML instances.........
2020-06-12 08:25:34 Starting - Preparing the instances for training...
2020-06-12 08:26:14 Downloading - Downloading input data...
2020-06-12 08:26:48 Training - Downloading the training image...
2020-06-12 08:27:03 Training - Training image download completed. Training in progress.2020-06-12 08:27:03,732 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-06-12 08:27:03,735 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-12 08:27:03,744 sagemaker_sklearn_container.training INFO     Invoking user training script.

2020-06-12 08:28:17 Uploading - Uploading generated training model
2020-06-12 08:28:17 Completed - Training job completed
2020-06-12 08:28:04,887 sagemaker-containers INFO     Module sklearn-boston-housing does not provide a setup.py. 
Generating setup.py
2020-06-12 08:2

In [6]:
import pandas as pd

data = pd.read_csv('housing.csv', delim_whitespace=True)
data.drop(['medv'], axis=1, inplace=True)
data.to_csv('data.csv', header=False, index=False)

batch_input = sess.upload_data(path='data.csv', key_prefix=prefix + "/batch")

In [7]:
sk_transformer = sk.transformer(instance_count=1, instance_type='ml.m5.large')

sk_transformer.transform(batch_input, content_type='text/csv', wait=True, logs=True)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


..................Processing /opt/ml/code
  Created wheel for sklearn-boston-housing: filename=sklearn_boston_housing-1.0.0-py2.py3-none-any.whl size=6031 sha256=27aef3388a747318a31bdba9d2a7cb5159063d88aec399118dd3b3efd1f3375e
  Stored in directory: /tmp/pip-ephem-wheel-cache-mg5w_fgc/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
Successfully built sklearn-boston-housing
/miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
[2020-06-12 08:31:26 +0000] [33] [INFO] Starting gunicorn 19.9.0
[2020-06-12 08:31:26 +0000] [33] [INFO] Listening at: unix:/tmp/gunicorn.sock (33)
[2020-06-12 08:31:26 +0000] [33] [INFO] Using worker: gevent
[2020-06-12 08:31:26 +0000] [36] [INFO] Booting worker with pid: 36
[2020-06-12 08:31:26 +0000] [38] [INFO] Booting worker with pid: 38

2020-06-12 08:3

In [16]:
print(sk_transformer.output_path)

s3://sagemaker-us-east-1-613904931467/sagemaker-scikit-learn-2020-06-12-08-28-30-978


In [21]:
%%bash -s "$sk_transformer.output_path"
aws s3 ls $1/
aws s3 cp $1/data.csv.out .
head -1 data.csv.out

2020-06-12 08:31:53      10909 data.csv.out
download: s3://sagemaker-us-east-1-613904931467/sagemaker-scikit-learn-2020-06-12-08-28-30-978/data.csv.out to ./data.csv.out
[[29.73828574177013], [24.920634119498292], [30.666100776465594], [28.695974897147465], [28.14313495300731], [25.131534815796883], [22.93779683063259], [19.675541704745292], [11.892192981811924], [19.02365348703733], [19.324158071572764], [21.51388611589652], [21.093656456568354], [19.481157567290104], [19.199715977483265], [19.20289016451153], [20.56523977980432], [16.97191229091579], [16.151575443182885], [18.249540155016803], [12.549099860265635], [17.61780918403315], [16.04338572589721], [13.890233364827616], [15.71557993966988], [13.375530219127672], [15.442910647261908], [14.89351409223272], [19.672129240011152], [21.06739077799732], [11.670072713244217], [17.96978006117805], [9.420959340186464], [14.26853500969085], [13.94905351382571], [23.476207520224055], [22.087165443555282], [22.911112182716003], [22.885798